# RAG with an Agent
* We will build a basic agent for doing retrieval when necessary and also holding a conversation.

In [1]:
%pip install -q faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### **Load**

In [2]:
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### **Retreiver**

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

#### **Retriever as Tool**

In [30]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "llm_powered_autonomous_agents",
    "Blog post about LLM Powered Autonomous Agents.",
)
tools = [tool]

#### **Prompt**

In [36]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)



prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[], template="You are a helpful assistant"
            )
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=["input"], template="{input}")
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [42]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [44]:
# question="hi, im Gus"
question="What are the approaches to Task Decomposition?"

result = agent_executor.invoke({"input": question})
result



> Entering new AgentExecutor chain...

Invoking: `llm_powered_autonomous_agents` with `{'query': 'approaches to Task Decomposition'}`


Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-s

{'input': 'What are the approaches to Task Decomposition?',
 'output': 'Task decomposition can be approached in several ways, particularly when utilizing Large Language Models (LLMs) and autonomous agents. Here are some key approaches:\n\n1. **Simple Prompting**: This involves using straightforward prompts to guide the LLM in breaking down tasks. Examples include asking for "Steps for XYZ" or "What are the subgoals for achieving XYZ?"\n\n2. **Task-Specific Instructions**: This method uses specific instructions tailored to the task at hand. For instance, prompting the LLM with "Write a story outline" for writing a novel.\n\n3. **Human Inputs**: Incorporating human feedback and insights can also aid in task decomposition, allowing for a more nuanced understanding of the task requirements.\n\n4. **LLM+P Approach**: This approach combines LLMs with classical planning techniques. It involves:\n   - Translating the task into a Planning Domain Definition Language (PDDL) format.\n   - Using a 

In [45]:
result = agent_executor.invoke({"input": 'What is my name?'})
result



> Entering new AgentExecutor chain...
I don't have access to personal information about users unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. If you'd like to share it, feel free!

> Finished chain.


{'input': 'What is my name?',
 'output': "I don't have access to personal information about users unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. If you'd like to share it, feel free!"}

#### **Adding in Memory**

In [47]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [48]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [49]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_with_chat_history.invoke(
    {"input": "hi! my name is Gus"},
    config={"configurable": {"session_id": "any"}},
)



> Entering new AgentExecutor chain...
Hi Gus! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is Gus',
 'chat_history': [],
 'output': 'Hi Gus! How can I assist you today?'}

In [ ]:
agent_with_chat_history.invoke(
    {"input": "What are the approaches to Task Decomposition?"},
    config={"configurable": {"session_id": "any"}},
)

In [50]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_with_chat_history.invoke(
    {"input": "Whats my name?"},
    config={"configurable": {"session_id": "any"}},
)



> Entering new AgentExecutor chain...
Your name is Gus! How can I help you today, Gus?

> Finished chain.


{'input': 'Whats my name?',
 'chat_history': [HumanMessage(content='hi! my name is Gus', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Gus! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Gus! How can I help you today, Gus?'}